# Elasticsearch Paper ID Explorer

Notebook này giúp liệt kê tất cả `paper_id` đang có trong index Elasticsearch và thống kê số lượng chunk.


## Nội dung
1. Cấu hình kết nối
2. Lấy danh sách paper_id (terms aggregation)
3. Lấy danh sách paper_id (scroll fallback)
4. Thống kê papers vs chunks
5. Xuất CSV

Chỉnh `ES_HOST` nếu cần (ví dụ `http://103.3.247.120:9202`).

In [6]:
# 1. Cấu hình kết nối
import os, json, math
from typing import List, Dict
import requests

ES_HOST = os.environ.get("ES_HOST", "http://103.3.247.120:9202")
INDEX = os.environ.get("ES_INDEX_NAME", "papers")
SESSION = requests.Session()
print(f"Using ES_HOST={ES_HOST}, INDEX={INDEX}")

# Helper
def es_get(path: str, body: Dict=None):
    url = f"{ES_HOST}/{path}" if not path.startswith('http') else path
    if body is None:
        r = SESSION.get(url, timeout=30)
    else:
        r = SESSION.post(url, json=body, timeout=60)
    r.raise_for_status()
    return r.json()

# Kiểm tra cluster
try:
    info = es_get("")
    print("Cluster name:", info.get("cluster_name"))
except Exception as e:
    print("Không kết nối được Elasticsearch:", e)

Using ES_HOST=http://103.3.247.120:9202, INDEX=papers
Cluster name: paper-search-cluster


In [7]:
# 2. Lấy danh sách paper_id bằng terms aggregation (nhanh, giới hạn top N)
TOP_N = 20  # đổi nếu cần
agg_body = {
    "size": 0,
    "query": {"term": {"doc_type": "paper"}},
    "aggs": {
        "papers": {
            "terms": {"field": "paper_id", "size": TOP_N}
        }
    }
}
try:
    agg_resp = es_get(f"{INDEX}/_search", agg_body)
    buckets = agg_resp.get('aggregations', {}).get('papers', {}).get('buckets', [])
    paper_ids_agg = [b['key'] for b in buckets]
    print(f"Agg mode: lấy được {len(paper_ids_agg)} paper_id (top {TOP_N})")
    paper_ids_agg[:10]
except Exception as e:
    print("Aggregation lỗi:", e)
    paper_ids_agg = []

Agg mode: lấy được 20 paper_id (top 20)


In [8]:
# 3. Scroll fallback để lấy toàn bộ (nếu số lượng lớn hơn terms size)
# Chỉ chạy nếu cần đầy đủ và index lớn.
USE_SCROLL = False  # chuyển True nếu muốn
SCROLL_SIZE = 500
all_paper_ids_scroll = []

if USE_SCROLL:
    first = es_get(f"{INDEX}/_search?scroll=1m", {
        "size": SCROLL_SIZE,
        "_source": ["paper_id"],
        "query": {"term": {"doc_type": "paper"}}
    })
    scroll_id = first.get('_scroll_id')
    hits = first.get('hits', {}).get('hits', [])
    while hits:
        for h in hits:
            pid = h.get('_source', {}).get('paper_id')
            if pid:
                all_paper_ids_scroll.append(pid)
        nxt = es_get("_search/scroll", {"scroll": "1m", "scroll_id": scroll_id})
        scroll_id = nxt.get('_scroll_id')
        hits = nxt.get('hits', {}).get('hits', [])
    print(f"Scroll mode: lấy được {len(all_paper_ids_scroll)} paper_id")
else:
    print("Bỏ qua scroll (USE_SCROLL=False)")

Bỏ qua scroll (USE_SCROLL=False)


In [9]:
# 4. Thống kê số chunk cho mỗi paper (top 20)
# Lưu ý: sử dụng terms agg + top_hits để đếm matching chunk docs.
chunk_count_body = {
    "size": 0,
    "query": {"term": {"doc_type": "chunk"}},
    "aggs": {
        "papers": {
            "terms": {"field": "paper_id", "size": 20},
            "aggs": {
                "cnt": {"value_count": {"field": "chunk_index"}}
            }
        }
    }
}
try:
    cc_resp = es_get(f"{INDEX}/_search", chunk_count_body)
    cc_buckets = cc_resp.get('aggregations', {}).get('papers', {}).get('buckets', [])
    top_chunk_stats = [
        {"paper_id": b['key'], "chunks": b['cnt']['value'], "docs": b['doc_count']} for b in cc_buckets
    ]
    print("Top 20 papers theo số chunk (value_count trên chunk_index):")
    for row in top_chunk_stats:
        print(row)
except Exception as e:
    print("Lỗi thống kê chunk:", e)
    top_chunk_stats = []

Top 20 papers theo số chunk (value_count trên chunk_index):
{'paper_id': '2508.00910', 'chunks': 465, 'docs': 465}
{'paper_id': '2210.14275', 'chunks': 425, 'docs': 425}
{'paper_id': '2508.21148', 'chunks': 403, 'docs': 403}
{'paper_id': '2508.19005', 'chunks': 278, 'docs': 278}
{'paper_id': '2503.09454', 'chunks': 249, 'docs': 249}
{'paper_id': '2509.02464', 'chunks': 235, 'docs': 235}
{'paper_id': '2508.19689', 'chunks': 232, 'docs': 232}
{'paper_id': '2509.02547', 'chunks': 229, 'docs': 229}
{'paper_id': '2509.02444', 'chunks': 227, 'docs': 227}
{'paper_id': '2405.14093', 'chunks': 190, 'docs': 190}
{'paper_id': '2508.20109', 'chunks': 183, 'docs': 183}
{'paper_id': '2508.07407', 'chunks': 166, 'docs': 166}
{'paper_id': '2508.20325', 'chunks': 166, 'docs': 166}
{'paper_id': '2509.01324', 'chunks': 166, 'docs': 166}
{'paper_id': '2508.20453', 'chunks': 155, 'docs': 155}
{'paper_id': '2508.19229', 'chunks': 154, 'docs': 154}
{'paper_id': '2509.01909', 'chunks': 154, 'docs': 154}
{'pap

In [10]:
# 5. Xuất danh sách paper_id ra CSV (kết hợp agg + scroll nếu có)
import csv, datetime
all_ids = set(paper_ids_agg) | set(all_paper_ids_scroll)
print(f"Tổng hợp được {len(all_ids)} paper_id")
output_file = f"paper_ids_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
with open(output_file, 'w', newline='', encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(["paper_id"])
    for pid in sorted(all_ids):
        w.writerow([pid])
print("Đã ghi:", output_file)

Tổng hợp được 20 paper_id
Đã ghi: paper_ids_20250921_071854.csv


In [11]:
# 6. Liệt kê paper_id cùng paper_title (top N)
TOP_N_TITLES = 20  # đổi nếu cần
try:
    title_body = {
        "size": TOP_N_TITLES,
        "query": {"term": {"doc_type": "paper"}},
        "_source": ["paper_id", "title", "paper_title"],
        "sort": ["_doc"]
    }
    title_resp = es_get(f"{INDEX}/_search", title_body)
    hits = title_resp.get('hits', {}).get('hits', [])
    rows = []
    for h in hits:
        src = h.get('_source', {})
        pid = src.get('paper_id')
        # Một số ingestion có thể dùng 'title' hoặc 'paper_title'
        ptitle = src.get('paper_title') or src.get('title') or '<NO_TITLE>'
        rows.append((pid, ptitle[:160]))
    print(f"Lấy {len(rows)} papers (paper_id, title):")
    for r in rows:
        print(r)
except Exception as e:
    print("Lỗi lấy paper titles:", e)

Lấy 20 papers (paper_id, title):
('2211.09623', 'Cross-Modal Adapter for Vision-Language Retrieval')
('2212.07126', 'Explainability of Text Processing and Retrieval Methods: A Survey')
('2301.06375', 'OLKAVS: An Open Large-Scale Korean Audio-Visual Speech Dataset')
('2303.08032', 'Verifying the Robustness of Automatic Credibility Assessment')
('2405.14314', 'Towards Efficient LLM Grounding for Embodied Multi-Agent Collaboration')
('2405.14862', 'Bitune: Leveraging Bidirectional Attention to Improve Decoder-Only LLMs')
('2405.15165', 'SoAy: A Solution-based LLM API-using Methodology for Academic Information Seeking')
('2402.14533', 'Whose LLM is it Anyway? Linguistic Comparison and LLM Attribution for GPT-3.5, GPT-4 and Bard')
('2403.01777', 'NPHardEval4V: Dynamic Evaluation of Large Vision-Language Models with Effects of Vision')
('2403.04931', 'A Survey on Human-AI Collaboration with Large Foundation Models')
('2404.01245', 'A Statistical Framework of Watermarks for Large Language Mod